In [4]:
%pip install ipywidgets
%pip install solara

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
     |████████████████████████████████| 139 kB 6.5 MB/s            
     |████████████████████████████████| 214 kB 118.4 MB/s            
     |████████████████████████████████| 2.3 MB 109.5 MB/s            
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECAT

In [7]:
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import Output

# Artist selection widget
artist_toggle_options = ['At most k songs', 'At least k songs', 'None of', ' ']
artist_toggle = widgets.Dropdown(
    options=artist_toggle_options,
    description='Artist2:',
    disabled=False,
    value=' '
)

# Number of songs for artist
artist_song_count = widgets.IntSlider(
    value=1,
    min=0,
    max=100,  # Adjust maximum as needed
    step=1,
    description='Songs Count:',
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
    layout=widgets.Layout(width='25%'),
    style={'description_width': 'initial'}
)

artist_name = widgets.Text(description='Artist name:')
song_name = widgets.Text(description='Song name:')

# artist_and_count = widgets.HBox([artist_toggle])
# artist_and_count.layout.align_items = 'center'

# intialize the output - second widget
out = Output()

def changed(change):
    '''
    Monitor change in the first widget
    '''
    global out

    print("test")

    if artist_toggle_options.value == 'At least k songs':
        out.append_display_data(artist_song_count)
        display(out)
        print("hi")
    elif artist_toggle_options.value == 'At most k songs':
        out.append_display_data(artist_song_count)
        display(out)
        print("hello")
    elif artist_toggle_options.value == 'None of':
        out.append_display_data(artist_name)
        display(out)
        print("a")
    else:
        out.clear_output()
        out = Output()
        print("b")

                
# monitor the friends widget for changes
artist_toggle.observe(changed, 'value')

# playlist_length_toggle = widgets.ToggleButtons(
#     description='Playlist length:',
#     options= ['Minutes', 'Number of Songs'])

# Playlist length in minutes
playlist_length_mins = widgets.IntRangeSlider(
    value=[0, 120],
    min=0,
    max=120,  # 2 hours in minutes
    step=10,  # step in minutes
    description='Playlist Length (min):',
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
    style={'description_width': 'initial'},
)

# # Playlist length in number of songs
# playlist_length_num_songs = widgets.IntSlider(
#     value=10,
#     min=2,
#     max=100,
#     step=1,
#     description='Playlist Length (number of songs):',
#     continuous_update=False,
#     orientation='horizontal',
#     readout=True,
#     readout_format='d',
#     style={'description_width': 'initial'},
# )

# # intialize the output - second widget
# out = Output()

# def changed(change):
#     '''
#     Monitor change in the first widget
#     '''
#     global out
#     if playlist_length_toggle.value == 'Minutes':  
#         out.append_display_data(playlist_length_mins)
#         # display(out)
#     elif playlist_length_toggle.value == 'Number of Songs':
#         out.append_display_data(playlist_length_num_songs)
#         # display(out)
#     else:
#         out.clear_output()
#         out = Output()
    
#     display(out)
#     out.clear_output()
                
# # monitor the friends widget for changes
# playlist_length_toggle.observe(changed, 'value')


# BPM range widget
bpm_range = widgets.IntRangeSlider(
    value=[60, 180],  # Default BPM range
    min=0,
    max=300,
    step=1,
    description='BPM Range:',
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

# Loudness widget
loudness_range = widgets.FloatRangeSlider(
    value=[-60.0, 0.0],
    min=-60.0,
    max=0.0,
    step=0.1,
    description='Loudness:',
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f'
)

# Liveness widget
liveness_range = widgets.FloatRangeSlider(
    value=[0.0, 1.0],
    min=0.0,
    max=1.0,
    step=0.01,
    description='Liveness:',
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f'
)

# Mode widget
mode_options = ['Major', 'Minor', 'Both']
mode_selection = widgets.Dropdown(
    options=mode_options,
    description='Mode:',
    disabled=False,
)

# Display widgets
display(artist_toggle, bpm_range, loudness_range, liveness_range, mode_selection)

Dropdown(description='Artist2:', index=3, options=('At most k songs', 'At least k songs', 'None of', ' '), val…

IntRangeSlider(value=(60, 180), continuous_update=False, description='BPM Range:', max=300)

FloatRangeSlider(value=(-60.0, 0.0), continuous_update=False, description='Loudness:', max=0.0, min=-60.0, rea…

FloatRangeSlider(value=(0.0, 1.0), continuous_update=False, description='Liveness:', max=1.0, step=0.01)

Dropdown(description='Mode:', options=('Major', 'Minor', 'Both'), value='Major')

In [8]:
import pandas as pd
import numpy as np

ModuleNotFoundError: No module named 'pandas'

In [ ]:
df = pd.read_csv('data/spotify_playlists.tsv', sep='\t', index_col=0)
df = df.sample(100)
# assign a random similarity score
df["similarity"] = np.random.rand(len(df))
df

In [ ]:
df.columns

In [ ]:
artist_and_count.children[0].value

In [ ]:
# constraints 
# 1) order by similarities 
# 2) apply contraints from widgets 
df = df.sort_values("similarity", ascending=False)
df = df[(df["tempo"] > bpm_range.value[0]) & (df["tempo"] < bpm_range.value[1])]
df = df[(df["loudness"] > loudness_range.value[0]) & (df["loudness"] < loudness_range.value[1])]
df = df[df["mode"] == mode_selection.value]
# todo: do all the range/equality contraints, 
# then do the "hard" ones like artist_song_count and playlist_length and num_song in that order probably 
# make sure artist_count <= num_songs
df

In [ ]:
df["tempo"].describe()

In [ ]:
filtering_criterion = {
    "Author": "SZA",
    "ms_played": lambda x: x < 100_000
}

In [146]:
import pandas as pd

# Assuming you have a dataframe named 'songs_df' with columns 'song_name', 'artist', 'duration_minutes'
# Example dataframe creation
data = {'song_name': ['Song1', 'Song2', 'Song3', 'Song4', 'Song5'],
        'artist': ['Bob', 'Alice', 'Bob', 'Charlie', 'Alice'],
        'duration_minutes': [3, 4, 2, 5, 3]}
songs_df = pd.DataFrame(data)

def select_songs(songs_df, playlist_length=None, artist_constraint=None, artist_name=None, artist_condition=None):
    # Constraint 3: Include the specified number of songs for the given artist and condition
    if artist_constraint and artist_name and artist_condition is not None:
        artist_songs = songs_df[songs_df['artist'] == artist_name]
        if artist_condition == 'at least':
            artist_songs = artist_songs.head(max(artist_constraint, len(artist_songs)))
        elif artist_condition == 'at most':
            artist_songs = artist_songs.head(min(artist_constraint, len(artist_songs)))
        elif artist_condition == 'none':
            artist_songs = pd.DataFrame(columns=songs_df.columns)
        else:
            print("Invalid artist condition.")
            return None
    else:
        artist_songs = pd.DataFrame(columns=songs_df.columns)

    # Constraint: Select songs until the playlist reaches the specified length +/- 10 minutes
    remaining_duration = max(0, playlist_length - artist_songs['duration_minutes'].sum())
    selected_songs = songs_df.head(len(artist_songs))
    while remaining_duration > 0:
        next_song = songs_df.iloc[len(selected_songs)]
        if abs(remaining_duration - next_song['duration_minutes']) <= 10:
            selected_songs = pd.concat([selected_songs, next_song.to_frame().transpose()])
            break
        elif remaining_duration - next_song['duration_minutes'] > 0:
            selected_songs = pd.concat([selected_songs, next_song.to_frame().transpose()])
            remaining_duration -= next_song['duration_minutes']
        else:
            break

    # Combine selected songs based on the order of constraints
    final_selection = pd.concat([artist_songs, selected_songs])

    return final_selection

# Example usage:
selected_subset = select_songs(songs_df, playlist_length=17, artist_constraint=2, artist_name='Bob', artist_condition='at least')
print(selected_subset)


  song_name artist duration_minutes
0     Song1    Bob                3
2     Song3    Bob                2
0     Song1    Bob                3
1     Song2  Alice                4
2     Song3    Bob                2


In [ ]:
%pip install python-constraint

In [ ]:
# FROM JESS'S Inference.ipynb

# function to get top-n most similar tracks
def get_most_similar_tracks(track_name, n=10, tokenizer=tokenizer, embedding_weights=embedding_weights):
    
    # get track embedding
    track_idx = tokenizer.word_index[track_name]
    track_vector = embedding_weights[track_idx, :].reshape(1, -1)

    # compute similarities against other tracks
    similarities = np.dot(track_vector, embedding_weights.T) / (np.linalg.norm(track_vector) * np.linalg.norm(embedding_weights, axis=1))
    similarities = similarities.reshape(-1)

    # get most similar tracks' indices
    most_similar_idxs = np.argpartition(similarities, -(n+1))[-(n+1):]
    most_similar_idxs = most_similar_idxs[np.argsort(similarities[most_similar_idxs])][::-1][1:]

    # print most similar tracks, along with their positions in training data
    print("top {} tracks most similar to '{}' (pos. {}):".format(n, track_name, track_idx))
    for idx in most_similar_idxs:
        print("- (sim. {:.3f}): '{}' (pos. {})".format(similarities[idx], tokenizer.index_word[idx], idx))